In [ ]:
import shutil
shutil.rmtree("examplemodel")

In [ ]:
from PIL import Image
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import cv2
import sys, re
import glob

In [ ]:
input_images = []
output_images = []
img_dir = str( "edges/" )

kernel_h = 3
kernel_w = 3
D = 1

def loadImages():
    global W
    global H
    global WH
    global input_images
    global output_images
    ins = []
    outs = []

    print("Loading images")
    filelist = glob.glob(os.path.join(img_dir, '*.jpg'))
    for imgFilename in sorted(filelist):
        if not imgFilename.endswith(".jpg"):
            continue
        a = cv2.imread(imgFilename,0)#1=color, 0=grayscale
        H, W = a.shape[:2]
        WH = int(W/2)
        left = np.asarray(a[0:H, 0:WH])
        righ = np.asarray(a[0:H, WH:W])
        ins.append(left)
        outs.append(righ)
        
    input_images  = np.asarray(ins, dtype=np.float32)
    output_images = np.asarray(outs, dtype=np.float32)

In this example, we're going to have a neural network learn to build the edge map.
<br>The training set consists of images with the original image on the left and the edge map on the right (see below)
<br>The edge map has been generated with a 3x3 vertical Sobel operator, which looks as follows:

In [ ]:
sobel = np.asarray([[1, 0, -1],[2,0,-2],[1,0,-1]])
print(str(sobel))

In [ ]:

one = np.asarray(cv2.imread(img_dir+"/train1.jpg",0), dtype=np.float32)
plt.imshow(one, cmap='gray')

In [ ]:
two = np.asarray(cv2.imread(img_dir+"/train2.jpg",0), dtype=np.float32)
plt.imshow(two, cmap='gray')

In [ ]:
def model_function(features, labels, mode):
  input = features["x"]
  input_layer = tf.reshape(input, [-1, H, H, D])
  conv = tf.layers.conv2d( inputs=input_layer, filters=D, kernel_size=[kernel_h, kernel_w], strides = 1, padding="same", activation=tf.nn.relu, use_bias=False )
  pred_output = tf.reshape(conv, [-1, H, H, D]);

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=pred_output)

  outputGround = tf.reshape(labels, [-1, H, H, D])
  loss = tf.losses.mean_squared_error( labels = outputGround, predictions=pred_output)
  tf.summary.scalar("loss", loss)
  tf.summary.image('input', input_layer, max_outputs=2)
  tf.summary.image('output', pred_output, max_outputs=2)
  tf.summary.image('real output', outputGround, max_outputs=2)

  if mode == tf.estimator.ModeKeys.TRAIN:
      optimizer = tf.train.AdamOptimizer(learning_rate=0.001, epsilon=0.01)
      gradients, variables = zip(*optimizer.compute_gradients(loss))
      gradients, _ = tf.clip_by_global_norm(gradients, 1.0)
      optimize = optimizer.apply_gradients(zip(gradients, variables), global_step=tf.train.get_global_step())
      return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=optimize)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(labels=labels, predictions=label)}
  return tf.estimator.EstimatorSpec( mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
estimator = tf.estimator.Estimator(model_fn=model_function, model_dir="examplemodel")

loadImages()
print("Train data shape: "+ str(np.shape(input_images))+ " type=", str(input_images.dtype))
print("Labels shape: "+ str(np.shape(output_images))+ " type=", str(output_images.dtype))
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input_images},
    y=output_images,
    batch_size=2,
    num_epochs=None,
    shuffle=False)

Lets look at the kernels as we train... after a single pass, all the variables are initialized and we can print the kernel.

In [ ]:
estimator.train( input_fn=train_input_fn, steps = 1)
kernel_one = estimator.get_variable_value("conv2d/kernel").reshape([kernel_h, kernel_w])
plt.imshow(kernel_one, cmap='gray')

In [ ]:
estimator.train( input_fn=train_input_fn, steps = 3000)

In [ ]:
kernel_two = estimator.get_variable_value("conv2d/kernel").reshape([kernel_h, kernel_w])
diff = kernel_two - kernel_one
f, axarr = plt.subplots(1,3,figsize=(10,10))
axarr[0].imshow(kernel_one, cmap='gray')
axarr[0].set_title('init')
axarr[1].imshow(diff, cmap='gray')
axarr[1].set_title('delta')
axarr[2].imshow(kernel_two, cmap='gray')
axarr[2].set_title('final')

The final kernel starts to look very much like the original sobel operator. Success!<p>Let's look at the results.

In [ ]:
img = np.asarray(cv2.imread(img_dir+"/train2.jpg",0), dtype=np.float32)
print(str(img.shape))
H, W = img.shape[:2]
edges = img[0:H,H:]
img = img[0:H,0:H]
print("Loaded "+str(img.shape))
test_data = img.reshape(1,H,H,D)
test_data = np.asarray(test_data[:,:], dtype=np.float32)
eval_input_fn = tf.estimator.inputs.numpy_input_fn( x={"x":test_data}, num_epochs=1, shuffle=False )
eval_results = estimator.predict(input_fn=eval_input_fn)
fin = eval_results.__next__()
out = fin.astype('uint8', casting='unsafe')
out = out.reshape((200,200))

f, axarr = plt.subplots(1,3, figsize=(20,20))
axarr[0].imshow(img, cmap='gray')
axarr[0].set_title('input')
axarr[1].imshow(out, cmap='gray')
axarr[1].set_title('output')
axarr[2].imshow(edges, cmap='gray')
axarr[2].set_title('edges')